In [12]:
! pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 58.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.6 MB/s eta 0:00:0000:0100:01m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 25.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.3 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 4.9 MB/s eta 0:00:000:00:0100:01
  Attempting u

In [13]:
import os
import shutil
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import pandas as pd
from ultralytics import YOLO
from PIL import Image, ImageEnhance
import random

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [14]:
original_dirs = {
    'Calculus': '/kaggle/input/oral-diseases/Calculus/Calculus',
    'Caries': '/kaggle/input/oral-diseases/Data caries/Data caries/caries augmented data set/preview',
    'Gingivitis': '/kaggle/input/oral-diseases/Gingivitis/Gingivitis',
    'Ulcers': '/kaggle/input/oral-diseases/Mouth Ulcer/Mouth Ulcer/Mouth_Ulcer_augmented_DataSet/preview',
    'Tooth Discoloration': '/kaggle/input/oral-diseases/Tooth Discoloration/Tooth Discoloration /Tooth_discoloration_augmented_dataser/preview',
    'Hypodontia': '/kaggle/input/oral-diseases/hypodontia/hypodontia'
}


In [15]:
image_paths, labels = [], []
for class_name, class_dir in original_dirs.items():
    for root, dirs, files in os.walk(class_dir):
        for file in files:
            if file.lower().endswith(('.jpg', '.jpeg', '.png')):
                image_paths.append(os.path.join(root, file))
                labels.append(class_name)

df = pd.DataFrame({'Image Path': image_paths, 'Label': labels})

In [16]:
df_majority = df[df['Label'] != 'Calculus']
df_minority = df[df['Label'] == 'Calculus']

df_minority_upsampled = resample(df_minority,
                                 replace=True,
                                 n_samples=len(df_majority) // 3, # زيادة عدد الصور
                                 random_state=42)

df = pd.concat([df_majority, df_minority_upsampled])

In [17]:
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['Label'], random_state=42)


In [18]:
def augment_image(image_path, save_dir, label):
    img = Image.open(image_path).convert("RGB")

    # تدوير
    img_rot = img.rotate(random.randint(-15, 15))
    img_rot.save(os.path.join(save_dir, label, f"aug_rot_{os.path.basename(image_path)}"))

    # Flip أفقي
    img_flip = img.transpose(Image.FLIP_LEFT_RIGHT)
    img_flip.save(os.path.join(save_dir, label, f"aug_flip_{os.path.basename(image_path)}"))

    # تعديل الإضاءة
    enhancer = ImageEnhance.Brightness(img)
    img_bright = enhancer.enhance(random.uniform(0.8, 1.2))
    img_bright.save(os.path.join(save_dir, label, f"aug_bright_{os.path.basename(image_path)}"))


In [19]:
base_dir = '/kaggle/working/yolo_cls'
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

for cls in df['Label'].unique():
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(val_dir, cls), exist_ok=True)

In [20]:
for _, row in train_df.iterrows():
    dst_path = os.path.join(train_dir, row['Label'], os.path.basename(row['Image Path']))
    shutil.copy(row['Image Path'], dst_path)

    # Augmentation إذا الفئة Calculus
    if row['Label'] == 'Calculus':
        augment_image(row['Image Path'], train_dir, row['Label'])

In [21]:
for _, row in val_df.iterrows():
    shutil.copy(row['Image Path'], os.path.join(val_dir, row['Label'], os.path.basename(row['Image Path'])))

print("✅ تم تجهيز البيانات مع Augmentation و Oversampling")

✅ تم تجهيز البيانات مع Augmentation و Oversampling


In [11]:
model = YOLO("yolov8l-cls.pt")  # نموذج أكبر لتحسين الدقة
model.train(
    data=base_dir,
    epochs=200,
    imgsz=320,       # زيادة دقة الصور
    batch=16,
    lr0=0.0005,      # Learning rate أقل
    patience=15,
    name='yolo_cls_oral_diseases_boosted',
    optimizer='Adam',
    label_smoothing=0.1
)

WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Ultralytics 8.3.177 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_cls, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=320, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0005, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo_cls_oral_diseases_boosted, nbs=64, nms=False, opset=

AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1111.3±384.0 MB/s, size: 34.7 KB)


train: Scanning /kaggle/working/yolo_cls/train... 12734 images, 0 corrupt: 100%|██████████| 12734/12734 [00:04<00:00, 2965.54it/s]


train: New cache created: /kaggle/working/yolo_cls/train.cache
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 618.4±226.6 MB/s, size: 33.1 KB)


val: Scanning /kaggle/working/yolo_cls/val... 2588 images, 0 corrupt: 100%|██████████| 2588/2588 [00:01<00:00, 2567.34it/s]

...

                   all      0.944          1
EarlyStopping: Training stopped early as no improvement observed in last 15 epochs. Best results observed at epoch 61, best model saved as best.pt.
To update EarlyStopping(patience=15) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



76 epochs completed in 3.002 hours.
Optimizer stripped from runs/classify/yolo_cls_oral_diseases_boosted/weights/last.pt, 72.6MB
Optimizer stripped from runs/classify/yolo_cls_oral_diseases_boosted/weights/best.pt, 72.6MB

Validating runs/classify/yolo_cls_oral_diseases_boosted/weights/best.pt...
Ultralytics 8.3.177 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
YOLOv8l-cls summary (fused): 54 layers, 36,192,262 parameters, 0 gradients, 98.7 GFLOPs
train: /kaggle/working/yolo_cls/train... found 12734 images in 6 classes ✅ 
val: /kaggle/working/yolo_cls/val... found 2588 images in 6 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 81/81 [00:08<00:00,  9.26it/s]
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all      0.945          1
Speed: 0.2ms preprocess, 2.7ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/yolo_cls_oral_diseases_boosted


ultralytics.utils.metrics.ClassifyMetrics object with attributes:

confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7c15502f5b10>
curves: []
curves_results: []
fitness: 0.9725656807422638
keys: ['metrics/accuracy_top1', 'metrics/accuracy_top5']
results_dict: {'metrics/accuracy_top1': 0.9451313614845276, 'metrics/accuracy_top5': 1.0, 'fitness': 0.9725656807422638}
save_dir: PosixPath('runs/classify/yolo_cls_oral_diseases_boosted')
speed: {'preprocess': 0.16559945904375556, 'inference': 2.7442023948969734, 'loss': 0.000517322254243965, 'postprocess': 0.00704226661946731}
task: 'classify'
top1: 0.9451313614845276
top5: 1.0

In [22]:
from ultralytics import YOLO
from sklearn.metrics import classification_report
import os
import glob

# تحميل الموديل بأفضل weights
model = YOLO("/kaggle/input/bestfile/tensorflow2/default/1/lastDenatalClassifcationbestfile.pt")

# مسار بيانات التحقق
val_dir = "/kaggle/working/yolo_cls/val"

# قراءة الصور والليبلز الحقيقية
val_images = glob.glob(os.path.join(val_dir, "**", "*.*"), recursive=True)
true_labels = [os.path.basename(os.path.dirname(p)) for p in val_images]

# التنبؤ بالليبلز
pred_labels = []
for img_path in val_images:
    results = model.predict(source=img_path, imgsz=320, verbose=False)
    pred_class = results[0].names[int(results[0].probs.top1)]
    pred_labels.append(pred_class)

# طباعة Classification Report
print(classification_report(true_labels, pred_labels))


                     precision    recall  f1-score   support

           Calculus       0.81      0.97      0.88       517
             Caries       1.00      0.99      0.99       476
         Gingivitis       0.95      0.76      0.84       470
         Hypodontia       1.00      0.99      0.99       250
Tooth Discoloration       0.99      0.99      0.99       367
             Ulcers       1.00      1.00      1.00       508

           accuracy                           0.95      2588
          macro avg       0.96      0.95      0.95      2588
       weighted avg       0.95      0.95      0.94      2588

